In [1]:
%cd ..

d:\Blockchain\data\orai\orchai_validator


In [2]:
from labeling.ETLProcessor import ETLProcessor
from labeling.utils import get_spark, query
from omegaconf import OmegaConf
import pyspark.sql.functions as F

In [3]:
spark = get_spark('./lib/postgresql-42.5.0.jar')

def sample(spark):
    config = OmegaConf.load("./config_small.yml")
    df = query(spark, **config.db)
    df.printSchema()
    max_height = df.agg(F.max(df.block_height)).collect()[0].asDict()['max(block_height)']
    df = df.filter(df.block_height < max_height)
    return df

In [4]:
df = sample(spark)

Successfully queried data from database
root
 |-- block_height: integer (nullable = true)
 |-- operator_address: string (nullable = true)
 |-- jailed: boolean (nullable = true)
 |-- status: string (nullable = true)
 |-- tokens: decimal(38,0) (nullable = true)
 |-- commission_rate: float (nullable = true)
 |-- delegator_shares: decimal(38,0) (nullable = true)
 |-- self_bonded: decimal(38,0) (nullable = true)
 |-- propose: boolean (nullable = true)
 |-- vote: boolean (nullable = true)



In [5]:
accept_rate = 0.1
concentration_level = 0.9
vote_score = 2
propose_score = 6
A = 9
B = 4
C = 2
D = 4

df = ETLProcessor.data_scoring(df, accept_rate, concentration_level, vote_score, propose_score, A, B, C, D)

------------------------------------------------
Successfully converted voting_power_score column
------------------------------------------------
------------------------------------------------
Successfully converted commission_score column
------------------------------------------------
------------------------------------------------
Successfully converted self_bonded_score column
------------------------------------------------
------------------------------------------------
Successfully converted vote_propose_score column
------------------------------------------------
------------------------------------------------
Sucessfully converted final_score
------------------------------------------------


In [6]:
df.printSchema()

root
 |-- block_height: integer (nullable = true)
 |-- operator_address: string (nullable = true)
 |-- jailed: integer (nullable = true)
 |-- status: string (nullable = true)
 |-- tokens: decimal(38,0) (nullable = true)
 |-- commission_rate: float (nullable = true)
 |-- delegator_shares: decimal(38,0) (nullable = true)
 |-- self_bonded: decimal(38,0) (nullable = true)
 |-- propose: integer (nullable = false)
 |-- vote: integer (nullable = false)
 |-- validators_count_per_block: long (nullable = true)
 |-- total_token_amt_per_block: decimal(38,0) (nullable = true)
 |-- total_self_bonded_amt_per_block: decimal(38,0) (nullable = true)
 |-- tokens_proportion: decimal(38,6) (nullable = true)
 |-- voting_power_score: double (nullable = true)
 |-- comission_score: double (nullable = true)
 |-- self_bonded_score: double (nullable = true)
 |-- vote_propose_score: double (nullable = true)
 |-- score: double (nullable = true)



In [7]:
df.write.csv("./data/orchai", header=True)

In [12]:
dp = spark.read.csv("./data/orchai", inferSchema=True, header=True)

In [13]:
dp.columns

['block_height',
 'operator_address',
 'jailed',
 'status',
 'tokens',
 'commission_rate',
 'delegator_shares',
 'self_bonded',
 'propose',
 'vote',
 'validators_count_per_block',
 'total_token_amt_per_block',
 'total_self_bonded_amt_per_block',
 'tokens_proportion',
 'voting_power_score',
 'comission_score',
 'self_bonded_score',
 'vote_propose_score',
 'score']